In [10]:
import numpy as np
import regex as re

In [3]:
def tokenize(words):
    '''Takes in a string and tokenizes it by whitespace,
and returns a list of tokens.
    '''
    return words.split(' ')

In [4]:
# A test of our first function
tokenize('hi i am brendan')

['hi', 'i', 'am', 'brendan']

It looks like it would work for simple sentences, but having different punctuations and cases would be a tough look. Let's see if it would fail with the sentence "Good, that is good."

In [5]:
tokenize('Good, I am good.')

['Good,', 'I', 'am', 'good.']

Issues are capitalization and punctuation. Let's try to fix those in the tokenize.better function

In [ ]:
def tokenize(words):
    '''Takes in a string and tokenizes it by whitespace,
and returns a list of tokens, but this time better, 
accounting for capitalization and punctuation.
    '''
    return words.split(' ')

In [7]:
'i am! brendan.'.replace('.','')

'i am! brendan'

In [18]:
def better_tokenize(words):
    '''
    The difference here is that I am using regex to get rid of all punctuation,
    and then making everything lowercase. Only then can I split based on the 
    whitespace. Basically the regex says sub everything that's not whitespace
    and alphanumeric with nothing.
    '''
    return re.sub(r'[^\w\s]', '', words ).lower().split(' ')


In [19]:
better_tokenize('brendan!matthys')

['brendanmatthys']